In [ ]:
from __future__ import annotations

from torch.utils.data import random_split

from mmpfn.datasets.petfinder_pawpularity import PawpularityDataset

import os 
import torch 
import numpy as np 
import pandas as pd

from sklearn.metrics import r2_score, root_mean_squared_error
from mmpfn.models.mmpfn_v2 import MMPFNRegressor
from mmpfn.models.mmpfn_v2.constants import ModelInterfaceConfig
from mmpfn.models.mmpfn_v2.preprocessing import PreprocessorConfig
from mmpfn.scripts_finetune_mm.finetune_tabpfn_main import fine_tune_mmpfn

/home/wall/works/research/MultiModalPFN/mmpfn/models/dino_v2/layers/swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
/home/wall/works/research/MultiModalPFN/mmpfn/models/dino_v2/layers/attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
/home/wall/works/research/MultiModalPFN/mmpfn/models/dino_v2/layers/block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")


In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"  # for debugging

In [3]:
data_path = os.path.join(os.getenv('HOME'), "works/research/MultiModalPFN/mmpfn/data/pawpularity")
dataset = PawpularityDataset(data_path)
# _ = dataset.get_images()
_ = dataset.get_embeddings()

Load embeddings from embeddings/pawpularity/pawpularity.pt


In [ ]:
r2_scores = []
for seed in range(5):
    torch.manual_seed(seed)
    train_len = int(len(dataset) * 0.8)
    test_len = len(dataset) - train_len

    train_dataset, test_dataset = random_split(dataset, [train_len, test_len])

    X_train = train_dataset.dataset.x[train_dataset.indices]
    y_train = train_dataset.dataset.y[train_dataset.indices]
    X_test = test_dataset.dataset.x[test_dataset.indices]
    y_test = test_dataset.dataset.y[test_dataset.indices]
    image_train = train_dataset.dataset.embeddings[train_dataset.indices]#.unsqueeze(1)
    image_test = test_dataset.dataset.embeddings[test_dataset.indices]#.unsqueeze(1)
        
    for i in range(X_train.shape[1]):
        col = X_train[:, i]
        col[np.isnan(col)] = np.nanmin(col) - 1
    for i in range(X_test.shape[1]):
        col = X_test[:, i]
        col[np.isnan(col)] = np.nanmin(col) - 1

    torch.cuda.empty_cache()

    save_path_to_fine_tuned_model = "./finetuned_mmpfn_airbnb.ckpt"

    fine_tune_mmpfn(
        # path_to_base_model="auto",
        save_path_to_fine_tuned_model=save_path_to_fine_tuned_model,
        # Finetuning HPs
        time_limit=60,
        finetuning_config={"learning_rate": 0.00001, "batch_size": 1, "max_steps": 100},
        validation_metric="rmse",
        # Input Data
        X_train=pd.DataFrame(X_train),
        image_train=image_train,
        y_train=pd.Series(y_train),
        categorical_features_index=None,
        device="cuda",  # use "cpu" if you don't have a GPU
        task_type="regression",
        # Optional
        show_training_curve=False,  # Shows a final report after finetuning.
        logger_level=0,  # Shows all logs, higher values shows less
        freeze_input=True,  # Freeze the input layers (encoder and y_encoder) during finetuning
        mixer_type='MGM', # MGM MGM+CQAM
        mgm_heads=8,
        cqam_heads=4,
    )

    # disables preprocessing at inference time to match fine-tuning
    no_preprocessing_inference_config = ModelInterfaceConfig(
        FINGERPRINT_FEATURE=False,
        PREPROCESS_TRANSFORMS=[PreprocessorConfig(name='none')]
    )

    # Evaluate on Test Data
    model_finetuned = MMPFNRegressor(
        model_path=save_path_to_fine_tuned_model,
        inference_config=no_preprocessing_inference_config,
        ignore_pretraining_limits=True,
        mixer_type='MGM', # MGM MGM+CQAM
        mgm_heads=8,
        cqam_heads=4,
    )

    clf_finetuned = model_finetuned.fit(X_train, image_train, y_train)
    r2 = root_mean_squared_error(y_test, clf_finetuned.predict(X_test, image_test))
    print("r2 (Finetuned):", r2)
    r2_scores.append(r2)

Fine-tuning Steps:   1%|          | 1/100 [00:00<?, ?it/s][2025-09-12 00:35:00,424] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps:   2%|▏         | 2/100 [00:02<04:03,  2.49s/it, Best Val. Loss=1, Best Val. Score=-0.000421, Training Loss=1.32, Val. Loss=1, Patience=50, Utilization=0, Grad Norm=inf][2025-09-12 00:35:02,813] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps:  89%|████████▉ | 89/100 [04:30<00:33,  3.07s/it, Best Val. Loss=0.978, Best Val. Score=0.0216, Training Loss=-0.606, Val. Loss=1.09, Patience=-36, Utilization=0, Grad Norm=19]  [2025-09-12 00:39:30,504] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps: 101it [05:06,  3.06s/it, Best Val. Loss=0.978, Best Val. Score=0.0216, Training Loss=-0.658, Val. Loss=1.09, Patience=-47, Utilization=0, Grad Norm=7.15]                         
[2025-09-12 00:40:04,192] INFO - Initial Validation Loss: 1.0004208087921143 Best Valida

r2 (Finetuned): 0.0264033961428628


Fine-tuning Steps:   1%|          | 1/100 [00:00<?, ?it/s][2025-09-12 00:40:14,883] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps:   2%|▏         | 2/100 [00:04<07:45,  4.75s/it, Best Val. Loss=1, Best Val. Score=-0.000274, Training Loss=1.35, Val. Loss=1, Patience=50, Utilization=0, Grad Norm=inf]


KeyboardInterrupt: 

In [ ]:
# get mean and std of accuracy scores
mean_r2 = np.mean(r2_scores)
std_r2 = np.std(r2_scores)
print("Mean R2:", mean_r2)
print("Std R2:", std_r2)